In [10]:
from ucscgenome import Genome
import pandas as pd
import numpy as np

In [2]:
def oneHotEncodeSequence(sequence):
    oneHotDimension = (len(sequence), 4)
    dnaAlphabet = {"A":0, "G":1, "C":2, "T":3}
    one_hot_encoded_sequence = np.zeros(oneHotDimension, dtype=np.int)
    for i, nucleotide in enumerate(sequence):
        if nucleotide.upper() in dnaAlphabet:
            index = dnaAlphabet[nucleotide.upper()]
            one_hot_encoded_sequence[i][index] = 1
    return one_hot_encoded_sequence

In [3]:
genomeName = "hg19"
genomeDir = "/home/eramamur/resources/genomes/hg19/"
hg19 = Genome(genomeName, cache_dir=genomeDir, use_web=False)

In [4]:
snp_data = pd.read_csv("/projects/pfenninggroup/machineLearningForComputationalBiology/eramamur_stuff/ad_variants_processing/snigdha_snp_list_unique_haploreg_hg19_positions.txt",
                               header=0,
                               sep='\t')

In [5]:
snp_data.head()

,CHR,haploreg_hg19_pos,LD_RSID,REF,ALT
0,chr10,11707563,rs34388456,GCCT,G
1,chr10,11707915,rs11257227,A,G
2,chr10,11714507,rs74347557,C,T
3,chr10,11714686,rs77892763,C,T
4,chr10,11717397,rs11257238,T,C


In [6]:
def getSequence(allele,chrom,position,left,right,genome):
    alleleLength = len(allele)
    zeroBasedPosition = position-1
    deductable = "right"
    for i in range(alleleLength-1):
        if deductable=="right":
            right-=1
            deductable="left"
        elif deductable=="left":
            left-=1
            deductable="right"
    
    left_sequence = hg19[chrom][zeroBasedPosition-left:zeroBasedPosition]
    right_sequence = hg19[chrom][zeroBasedPosition+alleleLength:zeroBasedPosition+alleleLength+right]
    
    sequence = left_sequence.lower()+allele.lower()+right_sequence.lower()
    return sequence

In [7]:
ref_sequences_1kb = []
alt_sequences_1kb = []
ref_sequences_500bp = []
alt_sequences_500bp = []
ref_sequences_501bp = []
alt_sequences_501bp = []

left_1kb = 499
right_1kb = 500
left_500bp = 249
right_500bp = 250
left_501bp = 250
right_501bp = 250

for index,row in snp_data.iterrows():
    ref = row["REF"]
    alt = row["ALT"]
    chrom = row["CHR"]
    position = row["haploreg_hg19_pos"]
    

    ref_sequence_1kb = getSequence(ref,chrom,position,left_1kb,right_1kb,hg19)
    alt_sequence_1kb = getSequence(alt,chrom,position,left_1kb,right_1kb,hg19)

    ref_sequence_500bp = getSequence(ref,chrom,position,left_500bp,right_500bp,hg19)
    alt_sequence_500bp = getSequence(alt,chrom,position,left_500bp,right_500bp,hg19)

    ref_sequence_501bp = getSequence(ref,chrom,position,left_501bp,right_501bp,hg19)
    alt_sequence_501bp = getSequence(alt,chrom,position,left_501bp,right_501bp,hg19)
    
    
    #assertions to check function definition
    refLength = len(ref)
    if refLength%2==0:
        midPoint = position - 2 + refLength//2
    else:
        midPoint = position - 1 + refLength//2
        
    hg19_ref_sequence_1kb = hg19[chrom][midPoint-left_1kb:midPoint+right_1kb+1].lower()
    assert(hg19_ref_sequence_1kb==ref_sequence_1kb)
 
    hg19_ref_sequence_500bp = hg19[chrom][midPoint-left_500bp:midPoint+right_500bp+1].lower()
    assert(hg19_ref_sequence_500bp==ref_sequence_500bp)

    ref_sequences_1kb.append(ref_sequence_1kb)
    alt_sequences_1kb.append(alt_sequence_1kb)
    
    ref_sequences_500bp.append(ref_sequence_500bp)
    alt_sequences_500bp.append(alt_sequence_500bp)
    
    ref_sequences_501bp.append(ref_sequence_501bp)
    alt_sequences_501bp.append(alt_sequence_501bp)

In [8]:
snp_data["ref_1kb"] = ref_sequences_1kb
snp_data["alt_1kb"] = alt_sequences_1kb
snp_data["ref_500bp"] = ref_sequences_500bp
snp_data["alt_500bp"] = alt_sequences_500bp
snp_data["ref_501bp"] = ref_sequences_501bp
snp_data["alt_501bp"] = alt_sequences_501bp

In [11]:
ref_501bp_output = "/projects/pfenninggroup/machineLearningForComputationalBiology/eramamur_stuff/ad_variants_processing/ref_501bp_sequences.npy"  
alt_501bp_output = "/projects/pfenninggroup/machineLearningForComputationalBiology/eramamur_stuff/ad_variants_processing/alt_501bp_sequences.npy"  

encodedRefSequences_501bp = []
encodedAltSequences_501bp = []

for index,row in snp_data.iterrows():
    refSequence_501bp = row["ref_501bp"]
    altSequence_501bp = row["alt_501bp"]
    
    encodedRefSequences_501bp.append(oneHotEncodeSequence(refSequence_501bp))
    encodedAltSequences_501bp.append(oneHotEncodeSequence(altSequence_501bp))
    
encodedRefSequences_501bp = np.stack(encodedRefSequences_501bp, axis=0)
encodedAltSequences_501bp = np.stack(encodedAltSequences_501bp, axis=0)

np.save(ref_501bp_output, encodedRefSequences_501bp)
np.save(alt_501bp_output, encodedAltSequences_501bp)

In [41]:
snp_data.to_csv("/projects/pfenninggroup/machineLearningForComputationalBiology/eramamur_stuff/ad_variants_processing/snigdha_snp_list_unique_haploreg_hg19_positions_1kb_and_500bp_sequences.txt",
                sep='\t',
                index=False)